In [21]:
import pandas as pd
import numpy as np
import re
import csv
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

np.random.seed(500)

In [22]:
df_prescriptions = pd.read_csv('C:/Users/harim/OneDrive/Documents/GitHub/dl4h_final_project_1/data/processed/ndc_codes_extracted.csv', dtype=str)

df_prescriptions.head()

,Unnamed: 0,NDC,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC
0,0,00469061711,Tacrolimus,Tacrolimus,Tacrolimus
1,1,00056017275,Warfarin,Warfarin,Warfarin
2,2,00338055002,Heparin Sodium,UNKNOWN,UNKNOWN
3,4,00054829725,Furosemide,Furosemide,Furosemide
4,5,00056016975,Warfarin,Warfarin,Warfarin


In [23]:

# ndc_codes = set()

# for index, record in df_prescriptions.iterrows():
#     ndc_codes.add(record['NDC'].strip())
        
# ndc_codes = list(ndc_codes)

# print(len(ndc_codes))

In [24]:
df = pd.read_csv('../data/processed/NOTE_EVENTS_WITH_DRUGS.csv', dtype=str)

# df = df.head(1000)

In [25]:
df['DRUGS_PRESENT_SPLIT'] = df['DRUGS_PRESENT'].str.split(pat=',') #.apply(lambda x: x.split(','))

df = df[df['DRUGS_PRESENT_SPLIT'].notnull()]

print("Count After Null Filter:", df.count())

Count After Null Filter: ROW_ID                 48589
SUBJECT_ID             48589
HADM_ID                48589
CHARTDATE              48589
CHARTTIME                  0
STORETIME                  0
CATEGORY               48589
DESCRIPTION            48589
CGID                       0
ISERROR                    0
TEXT                   48589
CLEAN_TEXT             48589
DRUGS_PRESENT          48589
DRUGS_PRESENT_SPLIT    48589
dtype: int64


In [26]:

free_text = df['CLEAN_TEXT']
label_codes = df['DRUGS_PRESENT_SPLIT']

### Text Vectorization

In [27]:

text_vectorizer = TfidfVectorizer(ngram_range=(3, 3),
                             analyzer='char')
X = text_vectorizer.fit_transform(free_text)

In [28]:
print(X.shape, text_vectorizer.get_feature_names())
print("....", X[0,0])

(48589, 28685) [' 0 ', ' 00', ' 01', ' 02', ' 03', ' 04', ' 05', ' 06', ' 07', ' 08', ' 09', ' 0a', ' 0b', ' 0c', ' 0d', ' 0e', ' 0f', ' 0g', ' 0h', ' 0i', ' 0j', ' 0l', ' 0m', ' 0n', ' 0o', ' 0p', ' 0r', ' 0s', ' 0t', ' 0u', ' 0v', ' 0w', ' 0x', ' 0z', ' 1 ', ' 10', ' 11', ' 12', ' 13', ' 14', ' 15', ' 16', ' 17', ' 18', ' 19', ' 1a', ' 1b', ' 1c', ' 1d', ' 1e', ' 1f', ' 1g', ' 1h', ' 1i', ' 1j', ' 1k', ' 1l', ' 1m', ' 1n', ' 1o', ' 1p', ' 1q', ' 1r', ' 1s', ' 1t', ' 1u', ' 1v', ' 1w', ' 1x', ' 1y', ' 2 ', ' 20', ' 21', ' 22', ' 23', ' 24', ' 25', ' 26', ' 27', ' 28', ' 29', ' 2a', ' 2b', ' 2c', ' 2d', ' 2e', ' 2f', ' 2g', ' 2h', ' 2i', ' 2k', ' 2l', ' 2m', ' 2n', ' 2o', ' 2p', ' 2q', ' 2r', ' 2s', ' 2t', ' 2u', ' 2v', ' 2w', ' 2x', ' 2y', ' 3 ', ' 30', ' 31', ' 32', ' 33', ' 34', ' 35', ' 36', ' 37', ' 38', ' 39', ' 3a', ' 3b', ' 3c', ' 3d', ' 3e', ' 3f', ' 3g', ' 3h', ' 3i', ' 3j', ' 3k', ' 3l', ' 3m', ' 3n', ' 3o', ' 3p', ' 3q', ' 3r', ' 3s', ' 3t', ' 3u', ' 3v', ' 3w', ' 3x', ' 3y

### Label Encoding

In [29]:
Y = np.array(label_codes)

In [30]:
from sklearn.preprocessing import MultiLabelBinarizer
    
lb = MultiLabelBinarizer()
Y = lb.fit_transform(Y)

# Y = Y.T

In [31]:
print("Labels found in first example: ", Y[0].sum())

print(Y[0])

Labels found in first example:  81
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1 1 1 0
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


In [32]:
print("X: ", X.shape, " Y: ", Y.shape)

X:  (48589, 28685)  Y:  (48589, 451)


In [33]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=109)

In [34]:
# from sklearn import svm
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression

# #Create a svm Classifier
# model = DecisionTreeClassifier(max_depth=5) # Linear Kernel

# #Train the model using the training sets
# model.fit(X_train, y_train)

# #Predict the response for test dataset
# y_pred = model.predict(X_test)

In [35]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=500, max_iter=100)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

c:\users\harim\appdata\local\programs\python\python36\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [36]:
print(y_pred[1])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


In [39]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred, average='micro'))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred, average='micro'))

Accuracy: 0.038691088701378884
Precision: 0.8629252735084724
Recall: 0.7539142602930297


In [40]:
import pickle
with open('vectorizer.pk', 'wb') as fin:
    pickle.dump(text_vectorizer, fin)

In [41]:
output = open('Departure_encoder.pkl', 'wb')
pickle.dump(lb, output)
output.close()

In [44]:
from joblib import dump, load

dump(model, 'model.joblib') 

['model.joblib']